注意注意注意！！！，本文提供的是txt文件类型的TCGA数据进行处理

声明：本文借鉴https://www.bilibili.com/video/BV117411Z7Cq?p=1 的操作方式，为该视频中操作方式的python复现

数据操作流程为：
- 数据下载
- 数据整理
- 数据融合
- 基因ID转换

本文B站视频连接[fakelink]("wentaoStudy.com")

本文代码Github仓库[fakelink2]("wentaostudy.com")

获取数据

TCGA数据下载来源：
> https://portal.gdc.cancer.gov/repository

基因名下载来源：
> https://www.gencodegenes.org/human/


基因名数据文件结构（本文使用的是v37版本，本文撰写时已经发布v38版本）
- gencode.v37.annotation.gff3

本文代码所需要的python工具包，可以使用pip进行安装
pandas、jupyter、numpy、gzip

数据整理，解压txt数据

In [6]:
#将下好的文件统一解压到指定目录
genefiledir = "gdc_download_20210522_014141.705327"
sampledir = "sampledir"
import gzip
import os
def un_gz(file_name , savename):
    
    # 获取文件的名称，去掉后缀名
    f_name = file_name.replace(".gz", "")
    # 开始解压
    g_file = gzip.GzipFile(file_name)
    #读取解压后的文件，并写入去掉后缀名的同名文件（即得到解压后的文件）
    open(savename , "wb+").write(g_file.read())
    g_file.close()

list2 = os.walk(genefiledir)
for root, dirs, files in list2: 
    for f in files: 
        if(str(f)[0] != "." and str(f)[-11:] == "FPKM.txt.gz"):
            un_gz(os.path.join(root, f) , os.path.join(sampledir, f.replace(".gz", "")))

数据融合

In [9]:
json_file = 'metadata.cart.2021-05-22.json'
import json
import pandas as pd
# 将数据加载到一个列表中
pop_data = None
f = open(json_file)
pop_data = json.load(f)
f.close()
file_name_table =[]
for i in pop_data:
    file_name_table.append([i['file_name'][:-3],i['associated_entities'][0]['entity_submitter_id']])
SumData = pd.DataFrame()
for i in file_name_table:
    df = pd.read_table(os.path.join(sampledir, i[0]) , names = ["genes" , i[1]] ,index_col = 0 , header = None)
    if(SumData.empty):
        SumData = df
    else:
        SumData = pd.merge(SumData, df, how='right', on='genes')
SumData.to_csv("data.csv")

基因ID转换

In [10]:
id_name = pd.read_table("gid_gname.csv" , sep=',' , index_col=0)
id_name.gene_id = id_name.gene_id.map(lambda x : (str(x).split("."))[0])
id_name.gene_name = id_name.gene_name.map(lambda x : (str(x).split("."))[0])
id_name.set_index('gene_id',inplace=True)

sample_data = pd.read_table("data.csv" , sep=',' , index_col=0)
sample_data['gene_id'] = sample_data.index.map(lambda x : (str(x).split("."))[0])
sample_data.set_index(["gene_id" ],inplace=True)
sample_data =  pd.merge(sample_data, id_name, how='left', on='gene_id')
sample_data.set_index(["gene_name" ],inplace=True)
sample_data = sample_data.dropna()
sample_data.T.to_csv("samples.csv")